In [1]:
from bs4 import BeautifulSoup
import urllib2

In [2]:
import pandas as pd
import datetime as dt
import pandas as pd, numpy as np, datetime
import jupyterNotebookModuleImport

In [4]:
def earnings_yahoo(dateToCheck, printURL = False):
    """Scape yahoo finance for earnings data for a single date."""

    #yahoo finance has the earnings data on a page which has a range of dates. A range is required for the page to load.
    #prepare the dates
    startDate =  dateToCheck + dt.timedelta(days=-6)
    endDate =  dateToCheck
    startDateStr = startDate.strftime("%Y-%m-%d")
    endDateStr = endDate.strftime("%Y-%m-%d")
    dateToCheckStr = dateToCheck.strftime("%Y-%m-%d")
    
    #prepare the URL for scraping.
    quote_page = 'https://finance.yahoo.com/calendar/earnings?from='+startDateStr+'&to='+endDateStr+'&day='+dateToCheckStr
    if printURL:
        print quote_page
    
    #open the URL using urllib and beautiful soup. parse the page an scrape for the particular table.
    page = urllib2.urlopen(quote_page)
    soup = BeautifulSoup(page, 'html.parser')
    name_box = soup.find('table', attrs={'class': 'data-table W(100%) Bdcl(c) Pos(r) BdB Bdc($c-fuji-grey-c)'})
    if name_box == None:
        return None
    table_body = name_box.find('tbody')

    #generate the dataset using the table if it was found.
    data = []
    rows = table_body.find_all('tr')
    for row in rows:
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        data.append([ele for ele in cols if ele]) # Get rid of empty values
    
    #generate the list of headers for the table.
    header = []
    cols = name_box.find('thead').find_all('tr')[0].find_all('span')
    cols = [ele.text.strip() for ele in cols]
    header.append([ele for ele in cols if ele])
    header = header[0]
    
    #convert to a dataframe for the data to be returned
    returnedData = pd.DataFrame(data, columns=header)
    returnedData['date'] = dateToCheck
    
    return returnedData

In [92]:
def earnings_fixTypes(earnings):
    '''correct the data types for the earnings dataset'''
    earnings['EPS Estimate'] = pd.to_numeric(earnings['EPS Estimate'], errors='coerce')
    earnings['Reported EPS'] = pd.to_numeric(earnings['Reported EPS'], errors='coerce')
    earnings['Surprise(%)'] = pd.to_numeric(earnings['Surprise(%)'], errors='coerce')
    earnings['Symbol'] = earnings['Symbol'].astype(str)
    return earnings

In [88]:
def earnings_yahoo_range(start, end, printURL = False, fixDataTypes=True):
    """Scape yahoo finance for earnings data between two dates."""

    #prepare dates
    startDate = dt.datetime.strptime(start, '%Y-%m-%d')
    endDate = dt.datetime.strptime(end, '%Y-%m-%d')

    earningsData = pd.DataFrame()
    
    #loop through each day in the date range and call earnings_yahoo to compile the data.
    for i in range(0,(endDate-startDate).days+1):
        dateToRetrieve = startDate + dt.timedelta(days=i)
        retrievedDF = earnings_yahoo(dateToRetrieve, printURL)
        if retrievedDF is not None:
            earningsData = earningsData.append(retrievedDF, printURL)
    
    earningsData['announce'] = ""
    earningsData.loc[earningsData['Earnings Call Time'] == "Before Market Open",'announce'] = "0:00"
    earningsData.loc[earningsData['Earnings Call Time'] == "After Market Close",'announce'] = "23:59"

    #remove the items that didnt match "Before Market Open / Close" -- note this could be fixed later by parsing the date str
    #for example, if it says "12PM" then we are ignoring the data
    earningsData = earningsData[earningsData['announce'] <> ""] 

    #convert the announcement date and the time to a "dt" value representing the time of the announcement. 
    #We assume an overnight announcement is at midnight. 
    earningsData['dtString'] = earningsData['date'].astype(str) + ' ' + earningsData['announce'].astype(str)
    earningsData['dt'] = pd.to_datetime(earningsData['dtString'], format='%Y-%m-%d %H:%M')
    
    #Drop the date and annouce columns. And dtString isnt required anymore
    earningsData = earningsData.drop(['dtString'],axis=1)
    earningsData = earningsData.drop(['date'],axis=1)
    earningsData = earningsData.drop(['announce'],axis=1)
    earningsData = earningsData.drop(['Earnings Call Time'],axis=1)
    earningsData = earningsData.drop(['Company'],axis=1)
    
    if fixDataTypes:
        earningsData = earnings_fixTypes(earningsData)
    return earningsData

In [89]:
start = '2018-02-01'
end = '2018-02-01'
earnings = earnings_yahoo_range(start,end)

In [6]:
def retrieveData_gFinance(intervalMins, periodDays, symbol):

    interval = intervalMins*60
    period = str(periodDays)+'d'
    url = "http://www.google.com/finance/getprices?q="+symbol+"&i="+str(interval)+"&p="+period+"&f=d,o,h,l,c,v"
    #print url
    #url ='https://www.google.com/finance/getprices?q=AAPL&i=86400&p=14d&f=d,o,h,l,c,v'
    
    x=list(np.array(pd.read_csv(url,skiprows=7,header=None)))
    date=[]
    newx = []
    for i in range(0,len(x)):
        if x[i][0][0]=='a':
            t= datetime.datetime.fromtimestamp(int(x[i][0].replace('a','')))
            date.append(t)
            newx.append(x[i])
        elif x[i][0][0]=='T':
            continue
        else:
            date.append(t+datetime.timedelta(minutes=int(x[i][0])*interval/60))
            newx.append(x[i])
    data1=pd.DataFrame(newx,index=date)
    data1.columns=['a','Open','High','Low','Close','Vol']
    data1['Symbol'] = symbol
    data1 = data1.drop(columns=["a"])
    return data1


In [18]:
xx = retrieveData_gFinance(intervalMins=30, periodDays=90, symbol="ABMD")


In [321]:
priceData = pd.DataFrame()
for index, row in earningsData.iterrows():
    print row['Symbol']
    symbol = row['Symbol']
    priceData_Symbol = retrieveData_gFinance(intervalMins=30, periodDays=90, symbol=symbol)
    priceData = priceData.append(priceData_Symbol)

ABMD
BABA
NCZ
GOOGL
MO
AMZN
AME
AMGN
AIV
APO
AAPL
APTV
ATHN
AN
BAX
BDC
BMS
BKH
BX
BSX
BRKS
BC
CPT
CRS
CASS
CI
CME
COP
CY
DCT
DECK
DSPG
EMN
ETN
EW
ESL
EXPO
FCFS
FBHS
GPRO
HSY
INGR
IP
KELYA
KMT
KRG
LAZ
LQDT
LPLA
MHO
MMP
MPC
MRLN
MMC
MA
MCK
MDC
MTH
MTX
MSI
MPLX
MSBF
MSCI
MSGN
MWA
NFG
NGVC
OSIS
PACB
PH
PFIS
PJC
POST
POWI
PBH
PBIP
QCRH
DGX
RDN
RL
RGS
RTEC
SIGI
AON
ARTW
CVX
CLX
XOM
JOUT
LYB
MSG
MAN
MRK
MINI
MFSF
NATH
PSX
PSXP
ROP
SAIA
SCCO
S
TWIN
VLP
VRTS
WFT
WY
CAFD
AIN
AOI
AMSC
APTI
ARNC
BMI
BAH
BMY
BWLD
CSWC
CPST
CTLT
CHD
CRUS
CLW
MOC
BAP
RE
FN
FTNT
GLAD
HES
LEG
MAC
MCY
MLAB
MTGE
MTSC
NOV
OCLR
ON
PTIE
PAHC
PINC
RBC
RGCO
SALT
SWKS
ACXM
ATGE
ACM
AGCO
AKAM
ALCO
AGN
AOSL
ABC
APC
AGTC
ARMK
ADM
AI
AHH
ARRY
ARW
APAM
ABG
ATO
AXTA
ACLS
BZH
BDX
TECH
BBOX
BLKB
CBT
CNC
CERN
CEVA
CMG
CFX
TCS
CUB
CMI
DHT
DNKN
EMR
ENTG
ESE
EXTR
FIS
GBL
IT
GM
GILD
GAIN
GPK
GPP
GHL
HCSG
HIW
HMN
IESC
JCOM
JKHY
KFRC
LH
LII
LPT
LITE
MTSI
MANH
MKL
MTCH
MDU
MCC
MCHP
MODN
MLI
MYGN
NANO
NTGR
NEWR
NEU
OAK
OLN
OI
GLT
PAYC
PXD
PAA


In [327]:
priceData.to_csv('priceData.csv', sep='\t')
earningsData.to_csv('earningsData.csv', sep='\t')

In [25]:
def scrapeData_stockSummary(symbol):
    quote_page = 'https://finance.yahoo.com/quote/'+symbol+'/'
    page = urllib2.urlopen(quote_page)
    soup = BeautifulSoup(page, 'html.parser')
    name_box = soup.find('div', attrs={'id': 'quote-summary'})
    table_body= name_box
    data = []
    rows = table_body.find_all('tr')
    for row in rows:
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        data.append([ele for ele in cols if ele]) # Get rid of empty values
    
    x = pd.DataFrame(data)
    x = x.transpose()
    x.columns = x.iloc[0]
    x = x.reindex(x.index.drop(0))
    x.insert(0,"Symbol", value=symbol)
    return x

In [26]:
data = scrapeData_stockSummary('AAPL')


In [37]:
stockMetaData = pd.DataFrame()
symbolList = earningsData['Symbol']
success = []
fail = []
for symbol in symbolList:
    print symbol
    try:
        meta = scrapeData_stockSummary(symbol=symbol)
        stockMetaData = stockMetaData.append(meta) 
        success.append(symbol)
    except:
        fail.append(symbol)

print 'success: ' + str(len(success)) + ' | ' + 'fail: ' + str(len(fail))

ABMD
BABA
NCZ
GOOGL
MO
AMZN
AME
AMGN
AIV
APO
AAPL
APTV
ATHN
AN
BAX
BDC
BMS
BKH
BX
BSX
BRKS
BC
CPT
CRS
CASS
CI
CME
COP
CY
DCT
DECK
DSPG
EMN
ETN
EW
ESL
EXPO
FCFS
FBHS
GPRO
HSY
INGR
IP
KELYA
KMT
KRG
LAZ
LQDT
LPLA
MHO
MMP
MPC
MRLN
MMC
MA
MCK
MDC
MTH
MTX
MSI
MPLX
MSBF
MSCI
MSGN
MWA
NFG
NGVC
OSIS
PACB
PH
PFIS
PJC
POST
POWI
PBH
PBIP
QCRH
DGX
RDN
RL
RGS
RTEC
SIGI
AON
ARTW
CVX
CLX
XOM
JOUT
LYB
MSG
MAN
MRK
MINI
MFSF
NATH
PSX
PSXP
ROP
SAIA
SCCO
S
TWIN
VLP
VRTS
WFT
WY
CAFD
AIN
AOI
AMSC
APTI
ARNC
BMI
BAH
BMY
BWLD
CSWC
CPST
CTLT
CHD
CRUS
CLW
MOC
BAP
RE
FN
FTNT
GLAD
HES
LEG
MAC
MCY
MLAB
MTGE
MTSC
NOV
OCLR
ON
PTIE
PAHC
PINC
RBC
RGCO
SALT
SWKS
ACXM
ATGE
ACM
AGCO
AKAM
ALCO
AGN
AOSL
ABC
APC
AGTC
ARMK
ADM
AI
AHH
ARRY
ARW
APAM
ABG
ATO
AXTA
ACLS
BZH
BDX
TECH
BBOX
BLKB
CBT
CNC
CERN
CEVA
CMG
CFX
TCS
CUB
CMI
DHT
DNKN
EMR
ENTG
ESE
EXTR
FIS
GBL
IT
GM
GILD
GAIN
GPK
GPP
GHL
HCSG
HIW
HMN
IESC
JCOM
JKHY
KFRC
LH
LII
LPT
LITE
MTSI
MANH
MKL
MTCH
MDU
MCC
MCHP
MODN
MLI
MYGN
NANO
NTGR
NEWR
NEU
OAK
OLN
OI
GLT
PAYC
PXD
PAA


In [47]:
stockMetaData.to_csv('stockMetaData.csv', sep='\t')

In [55]:
priceData = pd.read_csv('priceData.csv', sep='\t',index_col=0)
earningsData = pd.read_csv('earningsData.csv', sep='\t',index_col=0)
stockMetaData = pd.read_csv('stockMetaData.csv', sep='\t',index_col=0)